## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [12]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        # Verificación de las restricciones
        if len(codigo) != 4 or not codigo.isdigit():
            raise ValueError("El código debe ser un número de 4 dígitos.")
        if not (1 <= len(nombre) <= 100):
            raise ValueError("El nombre debe tener entre 1 y 100 caracteres.")
        if not (10 <= precio <= 10000):
            raise ValueError("El precio debe estar entre 10 y 10,000.")
        if not (0 <= len(tipo) <= 20):
            raise ValueError("El tipo debe tener entre 0 y 20 caracteres.")
        if not (0 <= cantidad <= 1000):
            raise ValueError("La cantidad en existencia debe estar entre 0 y 1000.")
        
        self._codigo = codigo
        self._nombre = nombre
        self._precio = precio
        self._tipo = tipo
        self._cantidad = cantidad

    @property
    def codigo(self):
        return self._codigo

    @property
    def nombre(self):
        return self._nombre

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if valor >= 10 and valor <= 10000:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if valor >= 0 and valor <= 1000:
            self._cantidad = valor
    
    @property
    def tipo(self):
        return self._tipo

    def valorTotal(self):
        return self._precio * self._cantidad

    def __str__(self):
        return f"Código: {self._codigo}, Nombre: {self._nombre}, Precio: {self._precio}, Tipo: {self._tipo}, Cantidad: {self._cantidad}, Valor Total: {self.valorTotal()}"

In [24]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
print (p1.nombre,p1.codigo)
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Coca Cola 0001
Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [14]:
class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        if len(descripcion) < 1 or len(descripcion) > 100:
            raise ValueError("La descripción debe tener entre 1 y 100 caracteres.")
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos

    def esAplicable(self, producto, cantidad):
        return (producto.codigo in self.codigos or producto.tipo in self.tipos)

    def calcularDescuento(self, producto, cantidad):
        raise NotImplementedError("Método calcularDescuento debe ser implementado en las subclases")

class OfertaDescuento(Oferta):
    def __init__(self, porcentaje, codigos=[], tipos=[]):
        super().__init__(descripcion=f"Descuento del {porcentaje}%", codigos=codigos, tipos=tipos)
        self.porcentaje = porcentaje

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return (producto.precio * cantidad) * (self.porcentaje / 100)
        return 0

class Oferta2x1(Oferta):
    def __init__(self, *args, **kwargs):
        descripcion = kwargs.pop('descripcion', 'Oferta 2x1')
        super().__init__(descripcion, *args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            pares = cantidad // 2
            return producto.precio * pares
        return 0

In [15]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [16]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, mode='r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # Salta la cabecera
            for row in reader:
                codigo, nombre, precio, tipo, cantidad = row
                producto = Producto(codigo, nombre, float(precio), tipo, int(cantidad))
                catalogo.agregar(producto)
        return catalogo

    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    def guardar(self, archivo):
        with open(archivo, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Codigo', 'Nombre', 'Precio', 'Tipo', 'Unidades'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        ofertas_aplicables = [oferta for oferta in self.ofertas if oferta.esAplicable(producto, cantidad)]
        return ofertas_aplicables[0] if ofertas_aplicables else None

    def calcularDescuento(self, producto, cantidad):
        oferta = self.buscarOferta(producto, cantidad)
        return oferta.calcularDescuento(producto, cantidad) if oferta else 0

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        return sum(producto.precio * producto.cantidad for producto in self.productos)

    def informe(self):
        tipos = {}
        for producto in self.productos:
            if producto.tipo not in tipos:
                tipos[producto.tipo] = {"unidades": 0, "valor_total": 0}
            tipos[producto.tipo]["unidades"] += producto.cantidad
            tipos[producto.tipo]["valor_total"] += producto.precio * producto.cantidad

        precio_promedio = self.valorTotal / self.cantidadUnidades if self.cantidadUnidades > 0 else 0
        
        informe = [
            "INFORME CATALOGO",
            f"Cantidad de productos: {self.cantidadProductos}",
            f"Cantidad de unidades: {self.cantidadUnidades}",
            f"Precio Promedio: $ {precio_promedio:.2f}",
            f"Valor total: $ {self.valorTotal:.2f}",
            "Tipos de productos: "
        ]

        for tipo, datos in tipos.items():
            precio_promedio_tipo = datos["valor_total"] / datos["unidades"] if datos["unidades"] > 0 else 0
            informe.append(f"  - {tipo}: {datos['unidades']}u x $ {precio_promedio_tipo:.2f}")

        informe.append("Ofertas:")
        for oferta in self.ofertas:
            informe.append(f"  - {oferta.descripcion}")

        return "\n".join(informe)

    def vender(self, producto, cantidad):
        producto_en_catalogo = self.buscar(producto.codigo)
        if producto_en_catalogo:
            if producto_en_catalogo.cantidad >= cantidad:
                producto_en_catalogo.cantidad -= cantidad
            else:
                raise ValueError(f"No hay suficiente cantidad de {producto.nombre}. Solo hay {producto_en_catalogo.cantidad} disponibles.")
        else:
            raise ValueError(f"El producto {producto.codigo} no existe en el catálogo.")

In [17]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
print (catalogo.cantidadUnidades)
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

97
Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [18]:
class Cliente: 
    def __init__(self, nombre, cuit):
        self._nombre = nombre
        self._cuit = cuit
        
    @property
    def nombre(self):
        return self._nombre
    
    @nombre.setter
    def nombre(self, valor):
        if 1 <= len(valor) <= 100:
            self._nombre = valor

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        if len(valor) == 13 and valor[2] == '-' and valor[11] == '-' and valor[:2].isdigit() and valor[3:11].isdigit() and valor[12].isdigit():
            self._cuit = valor

In [19]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [20]:
from datetime import date

class Factura:
    numero_factura = 100  # Número secuencial inicial

    @classmethod
    def ultimaFactura(cls, num):
        cls.numero_factura = num

    @classmethod
    def nuevoNumero(cls):
        cls.numero_factura += 1
        return cls.numero_factura

    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        self.catalogo = catalogo
        self.cliente = cliente
        self.items = []  # Lista de tuplas (producto, cantidad)
        self.subtotal = 0
        self.descuentos = 0

    def agregar(self, producto, cantidad):
        # Verifica si el producto ya está en la factura y suma la cantidad
        for i, (p, c) in enumerate(self.items):
            if p.codigo == producto.codigo:
                self.items[i] = (p, c + cantidad)
                self.actualizarTotales()
                self.catalogo.vender(producto, cantidad)
                return
        # Si no está, lo agrega como nuevo
        self.items.append((producto, cantidad))
        self.actualizarTotales()
        self.catalogo.vender(producto, cantidad)

    def actualizarTotales(self):
        self.subtotal = 0
        self.descuentos = 0
        for producto, cantidad in self.items:
            subtotal_item = producto.precio * cantidad
            self.subtotal += subtotal_item

            # Calcular el descuento correspondiente para el producto
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            self.descuentos += descuento
    
    @property
    def total(self):
        return self.subtotal - self.descuentos

    @property
    def cantidadProductos(self):
        return len(self.items)

    @property
    def cantidadUnidades(self):
        return sum(cantidad for _, cantidad in self.items)

    def imprimir(self):
        resultado = [
            f"Factura: {self.numero}",
            f"Fecha  : {date.today()}",
            f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})",
            ""
        ]
        
        for producto, cantidad in self.items:
            subtotal_item = producto.precio * cantidad
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            resultado.append(f"- {cantidad} {producto.nombre} x ${producto.precio:.2f} = ${subtotal_item:.2f}")
            if descuento > 0:
                oferta = self.catalogo.buscarOferta(producto, cantidad)
                if isinstance(oferta, OfertaDescuento):
                    resultado.append(f"      Descuento {oferta.porcentaje}% aplicado: -${descuento:.2f}")
                elif isinstance(oferta, Oferta2x1):
                    resultado.append(f"      Oferta 2x1 aplicada: -${descuento:.2f}")
        
        resultado.append(f"\nSubtotal:   ${self.subtotal:.2f}")
        resultado.append(f"Descuentos: ${self.descuentos:.2f}")
        resultado.append(f"-----------------------")
        resultado.append(f"TOTAL:      ${self.total:.2f}")

        return "\n".join(resultado)



In [21]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
print (f1.cantidadUnidades)
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

8
Prueba pasada exitosamente!


In [22]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
print (f1.descuentos)
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

3890.0
Prueba pasada exitosamente!
